In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely

import os 
import glob

In [1]:
# Defining classes for the dataframes
class AmenityPoint:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

class AmenityPolygon:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

In [4]:
# Read the building footprints data

buildingfootprints_gdf = gpd.read_file('manila_building_footprints.geojson')

buildingfootprints_gdf.head()

,id,geometry
0,0,"POLYGON ((120.95954 14.62190, 120.95945 14.621..."
1,1,"POLYGON ((120.96084 14.62342, 120.96095 14.623..."
2,2,"POLYGON ((120.96181 14.60838, 120.96172 14.608..."
3,3,"POLYGON ((120.96087 14.62855, 120.96103 14.628..."
4,4,"POLYGON ((120.96158 14.62105, 120.96162 14.621..."


In [20]:
# Load the Manila amenities data into a Geopandas dataframe
from shapely import wkt

manila_amenities_df = pd.read_csv('manila_amenities.csv')
manila_amenities_df['geometry'] = manila_amenities_df['geometry'].apply(wkt.loads)
manila_amenities_gdf = gpd.GeoDataFrame(manila_amenities_df, crs='epsg:4326')

# Separate into point and polygon dataframes
manila_amenities_polygon_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Polygon']
manila_amenities_point_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Point']
manila_amenities_multipoly_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'MultiPolygon']

# Append multipolygons to the polygon dataframe
manila_amenities_polygon_gdf = gpd.GeoDataFrame(pd.concat([manila_amenities_polygon_gdf, manila_amenities_multipoly_gdf], ignore_index=True))

# Reset point dataframe index
manila_amenities_point_gdf.reset_index(drop=True, inplace=True)

# Add a column to the polygon dataframe to store a list of Amenity Points within the polygon
manila_amenities_polygon_gdf['amenity_points'] = None

In [22]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []

    for j, point in manila_amenities_point_gdf.iterrows():
        try:
            if polygon['geometry'].intersects(point['geometry']):
                # Append the index of the current point
                points_within_polygon.append(j)
        except Exception as e:
            print(f"Error processing polygon {i} point {j}: {e}")
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [25]:
manila_amenities_polygon_gdf

,amenity,addr_city,name,x,y,geometry,amenity_points
0,education,Manila,University of the Philippines - Manila,120.983483,14.579005,"POLYGON ((120.98389 14.57888, 120.98391 14.578...","[174, 175]"
1,education,Manila,NaN,120.988929,14.603745,"POLYGON ((120.98881 14.60384, 120.98885 14.603...",[81]
2,education,Manila,Nihongo Center,120.988241,14.603513,"POLYGON ((120.98835 14.60348, 120.98845 14.603...",[80]
3,education,Manila,Lyceum of the Philippines University,120.977851,14.591553,"POLYGON ((120.97754 14.59183, 120.97788 14.592...","[106, 107]"
4,education,Manila,Doña Teodora Alonzo High School,120.980364,14.612760,"POLYGON ((120.98051 14.61235, 120.98019 14.612...",[]
...,...,...,...,...,...,...,...
1290,education,Manila,General Emilio Aguinaldo Integrated School,121.010565,14.586530,"MULTIPOLYGON (((121.01109 14.58616, 121.01116 ...",[]
1291,education,Manila,Gregoria de Jesus Elementary School,120.975264,14.627836,"MULTIPOLYGON (((120.97534 14.62806, 120.97527 ...",[1426]
1292,education,Manila,Ignacio Villamor High School,121.008567,14.576126,"MULTIPOLYGON (((121.00874 14.57633, 121.00883 ...",[]
1293,health,Manila,Chinese General Hospital and Medical Center,120.987957,14.626102,"MULTIPOLYGON (((120.98884 14.62588, 120.98885 ...","[102, 514, 515, 724, 989, 990, 991, 996, 1654]"


In [27]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe

# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

    for index, row in amenities_polygon_gdf.iterrows():
        # Add a node for the polygon
        amenities_network.add_node(polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row['name'], addr_city=row['addr_city'], amenity_points=row['amenity_points'])

    for index, row in amenities_point_gdf.iterrows():
        # Add a node for the point
        amenities_network.add_node(point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row['name'], addr_city=row['addr_city'], is_in_polygon=False)

In [ ]:
# This algorithm iterates through each polygon to connect amenities together into zones
# It connects amenity polygons and points of the same amenity type that are within a radius from the centroid of the current amenity polygon

# NOTABLE HYPERPARAMETERS:
# Amenities are connected based on a radius from the centroid of the polygon
# Aggregated polygon length (perimeter) is used to check if zone is sufficiently large  

def combine_contiguous_amenities_by_polygon(graph, max_distance, max_perimeter):
    for node in graph:
        if graph.node[node]['geometry'].geom_type == 'Polygon':
            other_node_distances = []
            contained_points = []
            for other_node in graph:
                if graph.node[other_node]['geometry'].geom_type == 'Polygon' or graph.node[other_node]['geometry'].geom_type == 'Multipolygon':
                    if node != other_node:
                        if graph.node[node]['amenity'] == graph.node[other_node]['amenity']:
                            # Check if the other node is within the max distance from the centroid of the current node
                            if graph.node[node]['geometry'].centroid.distance(graph.node[other_node]['geometry']) < max_distance:
                                other_node_distances.append((other_node, graph.node[node]['geometry'].centroid.distance(graph.node[other_node]['geometry'])))
                elif graph.node[other_node]['geometry'].geom_type == 'Point':
                    if graph.node[node]['geometry'].intersects(graph.node[other_node]['geometry']) and graph.node[other_node]['is_in_polygon'] == False:
                        contained_points.append(other_node)
            # Sort the other node distances by distance
            other_node_distances.sort(key = lambda x: x[1]) 

            # Get smallest distance from other node distances
            if len(other_node_distances) > 0:
                for new_node, new_distance in other_node_distances:
                    # Combine the polygons into a single zone
                    combined_node = shapely.ops.unary_union([graph.node[node]['geometry'], graph.node[new_node]['geometry']])

                    # Check if the combined polygon is sufficiently large
                    if combined_node.length < max_perimeter:
                        # Replace the current node with the combined node
                        graph.node[node]['geometry'] = combined_node
                        graph.node[node]['name'] = graph.node[node]['name'] + ', ' + graph.node[new_node]['name']
                        graph.node[node]['lat'] = combined_node.centroid.y
                        graph.node[node]['lon'] = combined_node.centroid.x
                        graph.node[node]['amenity_points'] = graph.node[node]['amenity_points'] + graph.node[new_node]['amenity_points']
                        # Remove the other node
                        graph.remove_node(new_node)
                        break
            # Add the contained points to the current node
            graph.node[node]['amenity_points'] = graph.node[node]['amenity_points'] + contained_points['point_index']
            for point in contained_points:
                point['is_in_polygon'] = True
            